<a href="https://colab.research.google.com/github/10udCryp7/TV-command-synthesis/blob/main/notebooks/Concat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data

In [ ]:
!gdown 1nuvnyEOrSUokE6KV-qcV-Mniakqecjwn

Downloading...
From (original): https://drive.google.com/uc?id=1nuvnyEOrSUokE6KV-qcV-Mniakqecjwn
From (redirected): https://drive.google.com/uc?id=1nuvnyEOrSUokE6KV-qcV-Mniakqecjwn&confirm=t&uuid=4701922c-5237-4299-bda6-fe79394e2045
To: /kaggle/working/trimmed_speech.zip
100%|██████████████████████████████████████| 27.1M/27.1M [00:01<00:00, 24.7MB/s]


In [ ]:
!unzip -q trimmed_speech.zip

In [ ]:
!gdown 19Bp6dE-puegzKmm1oIPf-KGXsLphTO7a

Downloading...
From: https://drive.google.com/uc?id=19Bp6dE-puegzKmm1oIPf-KGXsLphTO7a
To: /kaggle/working/output.csv
100%|██████████████████████████████████████| 1.29k/1.29k [00:00<00:00, 4.87MB/s]


In [ ]:
!gdown 121dcbCvDVzB22uelCeYkg4Fs8TVbQ1tp

Downloading...
From: https://drive.google.com/uc?id=121dcbCvDVzB22uelCeYkg4Fs8TVbQ1tp
To: /kaggle/working/100-samples.csv
100%|██████████████████████████████████████| 29.0k/29.0k [00:00<00:00, 35.8MB/s]


# Analysis

In [ ]:
metadata = pd.read_csv('100-samples.csv', index_col = 0)

In [ ]:
import pandas as pd

bad_voice = pd.read_csv('output.csv')

# Concat

In [ ]:
bad_voice

,id,segment,error_message
0,single_active_4ff7f40d,full,cant get start end
1,single_mix_d337321c,seg_0,cant get start end
2,chain_mix_79b56df5,full,cant get start end
3,single_mix_5143fc18,seg_3,cant get start end
4,single_mix_5143fc18,seg_9,cant get start end
5,single_mix_5143fc18,seg_10,cant get start end
6,single_active_7aa90624,full,cant get start end
7,single_mix_c6252315,seg_0,cant get start end
8,single_mix_c6252315,seg_1,cant get start end
9,single_mix_c6252315,seg_2,cant get start end


In [ ]:
import os
import json
from pydub import AudioSegment
import ast

def concat(input_folder, output_folder, metadata, bad_list):
    results = []
    for idx, row in metadata.iterrows():
        num_segments = row['num_segments']
        if num_segments != 0:
            file_name = f"{row['type']}_{row['id']}"
            if file_name in bad_list:
                print(f"skip {file_name}")
                continue

            type_segments = ast.literal_eval(row['type_segments'])  # dict {'0': 'non_active', '1': 'active', ...}

            combined = AudioSegment.silent(duration=0)
            annotations = []
            current_time_ms = 0

            for i in range(num_segments):
                file_path = os.path.join(input_folder, file_name, f'{file_name}_seg_{i}_trimmed.wav')
                if not os.path.exists(file_path):
                    print(f"Missing: {file_path}")
                    continue

                segment = AudioSegment.from_wav(file_path)
                duration_ms = len(segment)

                label = type_segments.get(str(i), "unknown")
                annotations.append({
                    "label": label,
                    "start": current_time_ms / 1000.0,  # giây
                    "end": (current_time_ms + duration_ms) / 1000.0
                })

                combined += segment
                current_time_ms += duration_ms

            # Tạo thư mục output cho từng file
            file_output_dir = os.path.join(output_folder, file_name)
            os.makedirs(file_output_dir, exist_ok=True)

            # Lưu file audio concat
            output_wav = os.path.join(file_output_dir, f"{file_name}_concat.wav")
            combined.export(output_wav, format="wav")

            # Lưu file JSON annotation
            output_json = os.path.join(file_output_dir, f"{file_name}.json")
            with open(output_json, "w", encoding="utf-8") as f:
                json.dump(annotations, f, indent=2, ensure_ascii=False)

            results.append({"wav": output_wav, "json": output_json})
            print(f"Saved: {output_wav}, {output_json}")

    return results


In [ ]:
res = concat(input_folder = "trimmed_speech", output_folder = "concat_speechs", metadata = metadata, bad_list = bad_voice['id'].values)

skip single_mix_d337321c
skip chain_mix_79b56df5
skip single_mix_5143fc18
Saved: concat_speechs/chain_mix_fa4ee976/chain_mix_fa4ee976_concat.wav, concat_speechs/chain_mix_fa4ee976/chain_mix_fa4ee976.json
Saved: concat_speechs/single_mix_445f45d6/single_mix_445f45d6_concat.wav, concat_speechs/single_mix_445f45d6/single_mix_445f45d6.json
Saved: concat_speechs/chain_mix_b21f4dee/chain_mix_b21f4dee_concat.wav, concat_speechs/chain_mix_b21f4dee/chain_mix_b21f4dee.json
Saved: concat_speechs/single_mix_2e9958c3/single_mix_2e9958c3_concat.wav, concat_speechs/single_mix_2e9958c3/single_mix_2e9958c3.json
Saved: concat_speechs/chain_mix_5a93f120/chain_mix_5a93f120_concat.wav, concat_speechs/chain_mix_5a93f120/chain_mix_5a93f120.json
Saved: concat_speechs/single_mix_966300b9/single_mix_966300b9_concat.wav, concat_speechs/single_mix_966300b9/single_mix_966300b9.json
Saved: concat_speechs/chain_mix_410ea272/chain_mix_410ea272_concat.wav, concat_speechs/chain_mix_410ea272/chain_mix_410ea272.json
Save